In [1]:
import os
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
from torchvision import transforms
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [2]:
class LateFusionNet(torch.nn.Module):
    def __init__(self, num_classes):
        super(LateFusionNet, self).__init__()
        # Separate convolutional layers for RGB and depth streams
        self.conv_rgb = torchvision.models.resnet18(pretrained=False)
        self.conv_rgb.fc = torch.nn.Identity()
        
        self.conv_depth = torchvision.models.resnet18(pretrained=False)
        self.conv_depth.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.conv_depth.fc = torch.nn.Identity()
        
        # Fully connected layers for fusion
        self.fc1 = torch.nn.Linear(512 * 2, 128)  # Concatenating features from both streams
        self.fc2 = torch.nn.Linear(128, num_classes)  # Adjust num_classes based on your task
        
    def forward(self, rgbd_inputs):
        rgb_inputs = rgbd_inputs[:,:3,:,:]
        depth_inputs = rgbd_inputs[:,3,:,:].unsqueeze(1)

        # Forward pass for RGB stream
        rgb_features = self.conv_rgb(rgb_inputs)
        # Forward pass for depth stream
        depth_features = self.conv_depth(depth_inputs)
        
        # Concatenate features from both streams
        combined_features = torch.cat((rgb_features, depth_features), dim=1)
        
        # Fully connected layers for fusion
        x = self.fc1(combined_features)
        x = torch.relu(x)
        x = self.fc2(x)
        return x


In [3]:
class rgbd_dataset(torch.utils.data.Dataset):
    def __init__(self, rgb_dataset, depth_dataset, transform=None):
        self.rgb_dataset = rgb_dataset
        self.depth_dataset = depth_dataset
        self.transform = transform

    def __len__(self):
        return len(self.rgb_dataset)

    def __getitem__(self, idx):
        rgb_img, rgb_label = self.rgb_dataset[idx]
        depth_img, _ = self.depth_dataset[idx]

        # Convert depth image to single-channel grayscale
        depth_img = torch.mean(depth_img, dim=0, keepdim=True)

        # Concatenate RGB and depth images along the channel dimension
        rgbd_img = torch.cat([rgb_img, depth_img], dim=0)

        # Apply transformations if provided
        if self.transform:
            rgbd_img = self.transform(rgbd_img)

        # Return the rgbd_img and the label from the RGB image
        return rgbd_img, rgb_label

In [4]:
def get_path(folder, case, ver):
    
    if case.split('_')[1] == 'angle':
        rgb_data_dir = os.path.join(folder, 'dataset', 'train', 'angle', 'rgb')
        depth_data_dir = os.path.join(folder, 'dataset', 'train', 'angle', 'depth')
        classes = os.listdir(rgb_data_dir)
        
    elif case.split('_')[1] == 'rad':
        rgb_data_dir = os.path.join(folder, 'dataset', 'train', 'rad', 'rgb')
        depth_data_dir = os.path.join(folder, 'dataset', 'train', 'rad', 'depth')
        classes = os.listdir(rgb_data_dir)
    
    save_path = os.path.join(folder, 'Weights', f'{case}_v{ver}.pth')
    
    return rgb_data_dir, depth_data_dir, classes, save_path

In [5]:
def get_model(case, classes):
    
    model = torchvision.models.resnet18(pretrained=False)
    model.fc = torch.nn.Linear(512, len(classes))

    if case.split('_')[0] == 'depth':
        model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

    # elif case.split('_')[0] == 'rgb':
    # model.conv1 = torch.nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        
    elif case.split('_')[0] == 'Ergbd':
        model.conv1 = torch.nn.Conv2d(4, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

    elif case.split('_')[0] == 'Lrgbd':
        model = LateFusionNet(len(classes))

    model.eval()
    
    return model

In [6]:
def get_data(rgb_data_dir, depth_data_dir, case):

  rgb_train_transfor = transforms.Compose([transforms.ToTensor(),
                                          transforms.ColorJitter(brightness=.5, hue=.1, saturation=[0.5, 1.5], contrast=[0.5, 1.5]),
                                          ])
  rgb_val_transform = transforms.Compose([transforms.ToTensor(),
                                                      ])
  depth_train_transfor = transforms.Compose([transforms.ToTensor(),
                                          transforms.Grayscale(num_output_channels=1),
                                        ])    
  depth_val_transform = transforms.Compose([transforms.ToTensor(), 
                                                      transforms.Grayscale(num_output_channels=1),
                                                      ])

  RandomCrop_transform = transforms.Compose([transforms.RandomCrop(240, padding=25)])

  rgb_ds_train = ImageFolder(rgb_data_dir, transform=rgb_train_transfor)
  depth_ds_train = ImageFolder(depth_data_dir, transform=depth_train_transfor)

  rgb_ds = ImageFolder(rgb_data_dir, transform=rgb_val_transform)
  depth_ds = ImageFolder(depth_data_dir, transform=depth_val_transform)

  train_size = int(0.9 * len(rgb_ds))
  val_size = len(rgb_ds) - train_size

  rgb_train_ds, rgb_val_ds = torch.utils.data.random_split(rgb_ds, [train_size, val_size])
  depth_train_ds = torch.utils.data.Subset(depth_ds, rgb_train_ds.indices)
  depth_val_ds = torch.utils.data.Subset(depth_ds, rgb_val_ds.indices)

  rgb_train_ds.dataset = rgb_ds_train
  depth_train_ds.dataset = depth_ds_train

  if case.split('_')[0] == 'rgb':
    train_ds = rgb_train_ds
    val_ds = rgb_val_ds
    train_ds.dataset.transform = transforms.Compose([train_ds.dataset.transform, RandomCrop_transform]) 
      
  if case.split('_')[0] == 'depth':
    train_ds = depth_train_ds
    val_ds = depth_val_ds
    train_ds.dataset.transform = transforms.Compose([train_ds.dataset.transform, RandomCrop_transform])
        
  elif 'rgbd' in case.split('_')[0]:
    train_ds = rgbd_dataset(rgb_train_ds, depth_train_ds, RandomCrop_transform)
    val_ds = rgbd_dataset(rgb_val_ds, depth_val_ds)
    
  return train_ds, val_ds
    

In [7]:
def evaluate(model, dataloader, device, loss_fn):
    
    model.eval()
    total_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    
    with torch.no_grad():
        for data in dataloader:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            total_loss += loss.item()
            
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_predictions += labels.size(0)

    average_loss = total_loss / len(dataloader)
    accuracy = correct_predictions / total_predictions * 100
    
    return average_loss, accuracy

def fit(epochs, model, train_dl, val_dl, device, opt, loss_fn):
    
    scheduler = ReduceLROnPlateau(opt, mode='min', factor=0.5, patience=3, verbose=True)
    best_val_accuracy = 0.0
    best_model_state = None
    no_improvement_count = 0
    
    for epoch in range(epochs):
        model.train() 
        train_loss = 0.0
        correct_predictions = 0
        total_predictions = 0
                
        for data in train_dl:
            inputs, labels = data # get the inputs
            inputs = inputs.to(device) # send them to device
            labels = labels.to(device) # forward + backward + optimize
            outputs = model(inputs) # forward pass
            loss = loss_fn(outputs, labels) # calculate the loss
            opt.zero_grad() # zero the parameter gradients
            loss.backward() # backpropagation
            opt.step() # update parameters
            train_loss += loss.item() # accumulate loss
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_predictions += labels.size(0)
            
        train_accuracy = correct_predictions / total_predictions * 100
        train_loss /= len(train_dl)
        val_loss, val_accuracy = evaluate(model, val_dl, device, loss_fn)

        log = "Epoch: {} | Training Loss: {:.4f} | Validation Loss: {:.4f} | Training Accuracy: {:.3f}% | Validation Accuracy: {:.3f}%".format(epoch+1, train_loss, val_loss, train_accuracy, val_accuracy)
        print(log)
           
        if val_accuracy  > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model_state = model.state_dict()
            no_improvement_count = 0
            print(f'model saved.')
        # else:
        #     no_improvement_count += 1
        #     if no_improvement_count >= 5:
        #         print(f'Early stopping triggered after {epoch+1} epochs.')
        #         break
        
        scheduler.step(val_loss)
        
    return (best_model_state)

In [8]:
def show_batch(dl, case):
    if 'rgbd' in case.split('_')[0]:
        for images, _ in dl:
            fig, ax = plt.subplots(figsize=(20,20))
            ax.set_xticks([]); ax.set_yticks([])
            ax.set_facecolor('white')

            rgb_imgs = images[:,:3,:,:]
            depth_imgs = images[:,3,:,:].unsqueeze(1)
            depth_imgs = depth_imgs.repeat(1, 3, 1, 1)

            rgbd_imgs = []
            for i in range(0, len(rgb_imgs), 2):
                rgbd_imgs.append(rgb_imgs[i])
                rgbd_imgs.append(depth_imgs[i])
                
            ax.imshow(make_grid(rgbd_imgs,10).permute(1,2,0))
            break
    else:
        for images, _ in dl:
            fig, ax = plt.subplots(figsize=(10,10))
            ax.set_xticks([]); ax.set_yticks([])
            ax.imshow(make_grid(images,10).permute(1,2,0))
            break

In [9]:
cases = ['rgb_angle', 'rgb_rad', 'depth_angle', 'depth_rad', 'Ergbd_angle', 'Ergbd_rad']
folders = ['PIH', 'Wire'] # os.listdir(main_folder)
cameras = ['515']
ver = 0

# hyper-parameters
batch_size = 64
lr = 1e-4
epochs = 25
loss_fn = torch.nn.CrossEntropyLoss()

for camera in cameras:
    print('------------------------------',camera,'------------------------------')
    
    main_folder = os.path.join('./', camera)
        
    for folder in folders:
        print('--------------------',folder,'--------------------')
        
        folder = os.path.join(main_folder, folder)
        
        for case in cases:
            print('----------',case,'----------')
            
            rgb_data_dir, depth_data_dir, classes, save_path = get_path(folder, case, ver)
            model = get_model(case, classes)
            train_ds, val_ds = get_data(rgb_data_dir, depth_data_dir, case)

            # dataloaders - creating batches and shuffling the data
            train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
            val_dl  = DataLoader(val_ds, batch_size=batch_size, shuffle=False)
            # device - cpu or gpu?
            device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
            # loss criterion
            opt = torch.optim.Adam(model.parameters(), lr=lr)
            # build our model and send it to the device
            model.to(device)
            
            best_model = fit(epochs, model, train_dl, val_dl, device, opt, loss_fn)
            
            torch.save(best_model, save_path)


------------------------------ 515 ------------------------------
-------------------- PIH --------------------
---------- rgb_angle ----------
Epoch: 1 | Training Loss: 2.8162 | Validation Loss: 2.8269 | Training Accuracy: 6.204% | Validation Accuracy: 6.500%
model saved.
Epoch: 2 | Training Loss: 2.7846 | Validation Loss: 2.7709 | Training Accuracy: 7.278% | Validation Accuracy: 7.833%
model saved.
Epoch: 3 | Training Loss: 2.7796 | Validation Loss: 2.7733 | Training Accuracy: 6.981% | Validation Accuracy: 7.333%
Epoch: 4 | Training Loss: 2.7686 | Validation Loss: 2.7594 | Training Accuracy: 7.148% | Validation Accuracy: 7.333%
Epoch: 5 | Training Loss: 2.7637 | Validation Loss: 2.7650 | Training Accuracy: 7.296% | Validation Accuracy: 7.667%
Epoch: 6 | Training Loss: 2.7591 | Validation Loss: 2.7652 | Training Accuracy: 8.278% | Validation Accuracy: 6.500%
Epoch: 7 | Training Loss: 2.7528 | Validation Loss: 2.7463 | Training Accuracy: 8.463% | Validation Accuracy: 7.500%
Epoch: 8 | 